<center><img src="./images/logo.png" style="width: 300px;"></center>

# **Fruit Quality Assessment Using Deep Learning**
This notebook demonstrates a complete workflow for classifying fresh and rotten fruits using deep learning.

## **Problem Statement**

A leading fruit supply company has approached **FruitScan Solutions** to develop an automated solution that can distinguish between fresh and rotten fruits in real-time using image input from their supply belt. The supplier, primarily dealing in apples, oranges, and bananas, faces challenges in ensuring consistent fruit quality, often resulting in customer complaints due to the inclusion of rotten fruits in purchased baskets. Manual sorting is slow, inconsistent, and not scalable. The company seeks to increase the speed and accuracy of their quality control process by integrating a computer vision-based system that can automate fruit classification and help streamline their distribution workflow

### **Importing Required Libraries**
Essential libraries for deep learning (PyTorch, torchvision), data handling, and image processing are imported. The device (CPU or GPU) is set for computation.

In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io
import os
import glob
from PIL import Image

import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()


True

### **Downloading and Preparing the Dataset**
   The fruit image dataset is downloaded using KaggleHub. The dataset contains labeled images of fresh and rotten fruits, organized into training and validation folders.


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sriramr/fruits-fresh-and-rotten-for-classification")

print("Path to dataset files:", path)

100%|██████████| 3.58G/3.58G [01:29<00:00, 42.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sriramr/fruits-fresh-and-rotten-for-classification/versions/1


### **Visualizing the Dataset**
   A sample image from the dataset is displayed to provide a visual understanding of the data.


<center><img src="./images/fruits.png" style="width: 600px;"></center>

### **Model Selection and Initialization**
   A pre-trained VGG16 model is loaded from torchvision. The base model's weights are frozen to leverage transfer learning, and the classifier is customized for the fruit classification task.


In [3]:
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights

weights = VGG16_Weights.DEFAULT
vgg_model = vgg16(weights=weights)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 78.8MB/s]


In [4]:
# Freeze base model
vgg_model.requires_grad_(False)
next(iter(vgg_model.parameters())).requires_grad

False

In [5]:
vgg_model.classifier[0:3]

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
)

### **Model Customization**
   The classifier part of the VGG16 model is modified to output predictions for six classes (fresh and rotten for each fruit type). Additional fully connected layers and activation functions are added.


In [6]:
N_CLASSES = 6

my_model = nn.Sequential(
    vgg_model.features,
    vgg_model.avgpool,
    nn.Flatten(),
    vgg_model.classifier[0:3],
    nn.Linear(4096, 500),
    nn.ReLU(),
    nn.Linear(500, N_CLASSES)
)
my_model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### **Setting Up Loss Function and Optimizer**
   The cross-entropy loss function is used for multi-class classification. The Adam optimizer is initialized to update the model parameters during training.


In [7]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(my_model.parameters())
my_model = torch.compile(my_model.to(device))

### **Data Transformations and Augmentation**
   Image transformations such as resizing, rotation, and horizontal flipping are applied to augment the training data and improve model generalization.


In [8]:
pre_trans = weights.transforms()
pre_trans

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [9]:
IMG_WIDTH, IMG_HEIGHT = (224, 224)

random_trans = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale=(.8, 1), ratio=(1, 1)),
    transforms.RandomHorizontalFlip(),
])

### **Custom Dataset Class**
   A custom PyTorch Dataset class is defined to load images and labels from the dataset folders, apply transformations, and prepare data for training and validation.


In [10]:
DATA_LABELS = ["freshapples", "freshbanana", "freshoranges", "rottenapples", "rottenbanana", "rottenoranges"]

class MyDataset(Dataset):
    def __init__(self, data_dir):
        self.imgs = []
        self.labels = []

        for l_idx, label in enumerate(DATA_LABELS):
            data_paths = glob.glob(data_dir + label + '/*.png', recursive=True)
            for path in data_paths:
                img = tv_io.read_image(path, tv_io.ImageReadMode.RGB)
                self.imgs.append(pre_trans(img).to(device))
                self.labels.append(torch.tensor(l_idx).to(device))


    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.imgs)

### **DataLoader Setup**
   PyTorch DataLoaders are created for both training and validation datasets to efficiently batch and shuffle data during model training and evaluation.


In [11]:
n = 32

In [12]:
train_path = os.path.join(path,'dataset/train/')
train_data = MyDataset(train_path)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

In [13]:
valid_path = os.path.join(path, 'dataset/test/')
valid_data = MyDataset(valid_path)
valid_loader = DataLoader(valid_data, batch_size=n, shuffle=False)
valid_N = len(valid_loader.dataset)

In [14]:
train_N, valid_N

(10901, 2698)

### **Model Training**
   The model is trained for several epochs using the training data. After each epoch, the model's performance is validated on the validation set to monitor progress and prevent overfitting.


In [15]:
epochs = 15

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch+1))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 1


W0507 19:34:32.644000 370 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


Train - Loss: 52.1945 Accuracy: 0.9473
Valid - Loss: 6.4263 Accuracy: 0.9722
Epoch: 2
Train - Loss: 26.3110 Accuracy: 0.9719
Valid - Loss: 5.0178 Accuracy: 0.9804
Epoch: 3
Train - Loss: 22.5769 Accuracy: 0.9761
Valid - Loss: 2.6496 Accuracy: 0.9900
Epoch: 4
Train - Loss: 14.5521 Accuracy: 0.9850
Valid - Loss: 4.9524 Accuracy: 0.9833
Epoch: 5
Train - Loss: 13.5720 Accuracy: 0.9855
Valid - Loss: 3.8268 Accuracy: 0.9896
Epoch: 6
Train - Loss: 14.5329 Accuracy: 0.9847
Valid - Loss: 3.0160 Accuracy: 0.9863
Epoch: 7
Train - Loss: 10.8092 Accuracy: 0.9895
Valid - Loss: 3.2797 Accuracy: 0.9885
Epoch: 8
Train - Loss: 12.0271 Accuracy: 0.9883
Valid - Loss: 1.4801 Accuracy: 0.9930
Epoch: 9
Train - Loss: 12.7056 Accuracy: 0.9872
Valid - Loss: 5.4825 Accuracy: 0.9822
Epoch: 10
Train - Loss: 10.5114 Accuracy: 0.9903
Valid - Loss: 3.8088 Accuracy: 0.9889
Epoch: 11
Train - Loss: 7.8565 Accuracy: 0.9919
Valid - Loss: 1.8540 Accuracy: 0.9926
Epoch: 12
Train - Loss: 8.3065 Accuracy: 0.9922
Valid - Loss: 

### **Fine-Tuning the Model**
   After initial training, the base VGG16 model is unfrozen to allow fine-tuning of all layers. The optimizer's learning rate is reduced, and the model is trained for additional epochs to further improve accuracy.


In [16]:
# Unfreeze the base model
vgg_model.requires_grad_(True)
optimizer = Adam(my_model.parameters(), lr=.0001)

In [17]:
epochs = 5

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch+1))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 1
Train - Loss: 3.3145 Accuracy: 0.9972
Valid - Loss: 2.3053 Accuracy: 0.9941
Epoch: 2
Train - Loss: 2.9193 Accuracy: 0.9978
Valid - Loss: 2.3634 Accuracy: 0.9937
Epoch: 3
Train - Loss: 3.1447 Accuracy: 0.9962
Valid - Loss: 1.4405 Accuracy: 0.9944
Epoch: 4
Train - Loss: 2.9512 Accuracy: 0.9971
Valid - Loss: 1.3282 Accuracy: 0.9956
Epoch: 5
Train - Loss: 2.3268 Accuracy: 0.9978
Valid - Loss: 1.0168 Accuracy: 0.9956


### **Model Evaluation**
   The final trained model is evaluated on the validation set to assess its classification performance. Metrics such as accuracy and loss are reported.

In [18]:
utils.validate(my_model, valid_loader, valid_N, loss_function)

Valid - Loss: 1.0168 Accuracy: 0.9956


### **Testing the Model on New Images**
The following function allows you to test the trained fruit classification model on any new image. It loads an image, applies the necessary preprocessing, and predicts whether the fruit is fresh or rotten. You can use this function to evaluate the model's performance on unseen data or your own images.

In [19]:
import torch
import torchvision.transforms.v2 as transforms
from PIL import Image

def test_model(model, image_path):

    # Load and preprocess the image
    img = Image.open(image_path).convert("RGB")
    pre_trans = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    img_tensor = pre_trans(img).unsqueeze(0).to(device)

    # Make a prediction
    with torch.no_grad():
      model.eval()
      output = model(img_tensor)

    # Get the predicted class
    _, predicted = torch.max(output, 1)
    predicted_class = DATA_LABELS[predicted.item()]

    print(f"Predicted class for {image_path}: {predicted_class}")

In [ ]:
# Example usage
# image_path = "/content/images/pic1.png"
image_path = "/content/images/pic2.png"
# image_path = "/content/images/pic3.png"
test_model(my_model, image_path)


Predicted class for /content/images/pic2.png: rottenbanana


## **Saving the Model**
Finally the Fine-tuned model is saved in the `model` directory.

In [27]:
# Create the directory if it doesn't exist
!mkdir -p model

# Save the model
torch.save(my_model, 'model/fruit_classification_model.pth')
print("Model saved successfully in the model folder.")

Model saved successfully in the model folder.
